In [1]:
!ls ../../../models/

robertuito-lince-pos  robertuito-lince-sentiment  robertuito-lince-sentiment-2


In [4]:
%load_ext autoreload
%autoreload 2
from transformers import AutoModelForTokenClassification, AutoTokenizer
from pysentimiento.lince.pos import load_datasets

model_path = "../../../models/robertuito-lince-pos"

model = AutoModelForTokenClassification.from_pretrained(model_path)


tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.model_max_length = 128

_, _, test_dataset = load_datasets(lang="es", preprocess=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Reusing dataset lince (/root/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-135d204f96f66baa.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-0ada48fff114da4c.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-94a4a8128eaebf6a.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-b25ee59dae025001.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/lince/pos_spaeng/1.0.0/10d41747f55f0849fa84ac579ea1acfa7df49aa2015b60426bc459c111b3d589/cache-7420b238145cf866.arrow
Loading cached processed dataset at /root/.cache/huggingface/data

In [5]:
from datasets import Dataset
from pysentimiento.lince.pos import preprocess_token, tokenize_and_align_labels

"""
Hago esto porque necesito las palabras originales
"""


original_words = test_dataset["words"]

test_dataset = test_dataset.map(
    lambda x: {
        "words": [preprocess_token(word, "es") for word in x["words"]]
    }
)


tokenize_fun = lambda x: tokenize_and_align_labels(x, tokenizer)
test_dataset = test_dataset.map(
    tokenize_fun, batched=True, batch_size=32, 
)

test_dataset = test_dataset.remove_columns(["labels"])

  0%|          | 0/10720 [00:00<?, ?ex/s]

  0%|          | 0/335 [00:00<?, ?ba/s]

In [6]:
from tqdm.auto import tqdm

problematic_instances = []

for idx, row in tqdm(enumerate(test_dataset), total=len(test_dataset)):
    word_ids = row["word_ids"]
    input_ids = row["input_ids"]
    words = original_words[idx]
    if not (word_ids[-2] + 1) == len(words):
        problematic_instances.append(idx)

print(f"{len(problematic_instances)} problematic instances")
assert len(problematic_instances) == 0

  0%|          | 0/10720 [00:00<?, ?it/s]

0 problematic instances


In [7]:
from transformers import DataCollatorForTokenClassification, Trainer, TrainingArguments
data_collator = DataCollatorForTokenClassification(tokenizer)

train_args = TrainingArguments(
    output_dir="./test/",
    per_device_eval_batch_size=32,
    do_train=False,
)

trainer_args = {
    "model": model,
    "args": train_args,
    "eval_dataset": test_dataset,
    "data_collator": data_collator,
    "tokenizer": tokenizer,
}

eval_trainer = Trainer(**trainer_args)
ret = eval_trainer.predict(test_dataset)

The following columns in the test set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: idx, lid, word_ids, pos, words.
***** Running Prediction *****
  Num examples = 10720
  Batch size = 32


In [16]:
from tqdm.auto import tqdm
from pysentimiento.lince.pos import id2label, label2id

outputs = []

for idx, row in tqdm(enumerate(test_dataset), total=len(test_dataset)):
    word_ids = row["word_ids"]
    input_ids = row["input_ids"]
    words = original_words[idx]
    assert (word_ids[-2] + 1) == len(words)
    preds = ret.predictions[idx]

    sentence_output = []
    current_word_id = None

    sentence_output = [None] * len(words)

    for word_id, token_id, pred in zip(word_ids, input_ids, preds):
        token = tokenizer.decode(token_id)
        if current_word_id != word_id and word_id is not None:
            current_word_id = word_id
            label = id2label[pred.argmax()]
            # FIX: Split "B-"
            label = label.split("-")[-1]
            sentence_output[current_word_id] = label
    
    if not all(sentence_output):
        print("="*80)
        print(idx)
        print(list(zip(
            words,
            sentence_output,
        )))

        # Filling gaps
        sentence_output = [s or 'O' for s in sentence_output]
        
    outputs.append(sentence_output)


  0%|          | 0/10720 [00:00<?, ?it/s]

Toda pura mierda

In [17]:
def write_list(path, data):
    with open(path, "w") as f:
        for sentence in data:
            for row in sentence:
                f.write(f"{row}\n")
            f.write("\n")

write_list("../submissions/01_robertuito/pos_spa_eng.txt", outputs)

In [62]:
import torch
sum(map(torch.numel, model.parameters()))

108212755